In [747]:
# Basic Package
import pandas as pd
import numpy as np

# Stationary Test Package
from statsmodels.tsa.stattools import coint
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from hurst import compute_Hc
from statsmodels.tsa.stattools import adfuller

# Unsupervised learning Appraoches
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AffinityPropagation, DBSCAN, OPTICS
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Package for Observation
import matplotlib.pyplot as plt
import seaborn as sns

# Optimization Package
import pymoo

# For NSGA-II
#import model
from pymoo.algorithms.moo.nsga2 import NSGA2
#import problem
from pymoo.core.problem import ElementwiseProblem
#import minimizer
from pymoo.optimize import minimize

In [748]:
# Extract data from the source file with only close price
data = pd.read_csv("S&P500_Stock.csv", index_col = 0)
data

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,65.095131,51.647560,99.734367,40.888062,77.285172,86.405487,53.904888,29.433332,142.543961,177.699997,...,154.734467,41.177719,64.879845,64.099930,63.911427,74.335281,115.880241,103.709999,43.878445,69.429123
2018-01-03,66.751389,51.014030,100.636848,40.880947,78.494598,86.727074,54.024090,29.459999,143.201859,181.039993,...,153.058090,40.902119,66.154076,63.880337,64.690742,74.271515,116.683548,105.769997,43.826511,69.748352
2018-01-04,66.250664,51.335663,104.350220,41.070835,78.046959,86.534111,53.932404,29.570000,144.897491,183.220001,...,153.886826,40.583443,66.245644,63.870785,65.122620,75.027611,116.515419,107.860001,44.008263,70.164337
2018-01-05,67.309898,51.316174,105.459518,41.538448,79.405594,87.581573,54.088280,29.453333,146.092758,185.339996,...,154.913391,40.299225,66.192230,64.768211,65.000565,75.464859,117.673660,109.540001,44.025570,70.967278
2018-01-08,67.454346,50.809345,104.716873,41.384144,78.133331,89.033318,53.932404,29.456667,147.260239,185.039993,...,152.850906,40.600658,66.489830,65.207420,65.235306,75.592407,117.897827,110.629997,43.809204,71.818565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-10,135.720001,15.460000,128.039993,148.500000,149.710007,149.600006,96.959999,66.610001,252.949997,329.299988,...,108.339996,62.716564,107.779999,36.930000,98.980003,124.580002,123.529999,288.190002,40.349998,161.529999
2023-03-13,136.690002,14.850000,NaN,150.470001,151.949997,148.369995,98.190002,NaN,NaN,324.269989,...,106.250000,64.780006,106.540001,NaN,97.900002,NaN,124.250000,NaN,29.969999,NaN
2023-03-14,138.399994,14.660000,121.699997,152.589996,153.850006,149.289993,98.550003,67.339996,252.479996,333.329987,...,108.330002,65.440002,106.940002,38.130001,99.900002,126.699997,126.629997,292.459991,31.309999,164.559998


In [749]:
# Drop all the data that with missing data > 0.2
print('Data Shape before cleaning =', data.shape)

missing_percentage = data.isnull().mean().sort_values(ascending=False)
dropped_list = sorted(list(missing_percentage[missing_percentage > 0.2].index))
data.drop(labels=dropped_list, axis=1, inplace=True)
data = data.fillna(method='ffill')
data = data.fillna(method='bfill')
print('Data Shape after cleaning =', data.shape)

Data Shape before cleaning = (1310, 503)
Data Shape after cleaning = (1310, 494)


In [750]:
#cumulative return
cumret = np.log(data).diff().cumsum() + 1
cumret.loc[:,list] = cumret.loc[:,list].fillna(method='ffill')
cumret.loc[:,list] = cumret.loc[:,list].fillna(method='bfill')
print(len(cumret.values[:,0]))

1310


In [751]:
def cadf_pvalue(s1, s2, cumret):
    '''
    perform CADF cointegration tests
    since it is sensitive to the order of stocks in the pair, perform both tests (s1-s2 and s2-s1)
    return the smallest p-value of two tests
    '''
    p1 = coint(cumret[s1], cumret[s2])[1]
    p2 = coint(cumret[s2], cumret[s1])[1]
    
    return min(p1,p2)

In [752]:
from pymoo.core.sampling import Sampling
class MySampling(Sampling):

    def _do(self, problem, n_samples, **kwargs):
        X = np.full((n_samples, problem.n_var), False, dtype=bool)

        for k in range(n_samples):
            I = np.random.permutation(problem.n_var)[:problem.n_max]
            X[k, I] = True

        return X

In [753]:
from pymoo.core.crossover import Crossover

class BinaryCrossover(Crossover):
    def __init__(self):
        super().__init__(2, 1)

    def _do(self, problem, X, **kwargs):
        n_parents, n_matings, n_var = X.shape

        _X = np.full((self.n_offsprings, n_matings, problem.n_var), False)

        for k in range(n_matings):
            p1, p2 = X[0, k], X[1, k]

            both_are_true = np.logical_and(p1, p2)
            _X[0, k, both_are_true] = True

            n_remaining = problem.n_max - np.sum(both_are_true)

            I = np.where(np.logical_xor(p1, p2))[0]

            S = I[np.random.permutation(len(I))][:n_remaining]
            _X[0, k, S] = True

        return _X

In [754]:
from pymoo.core.mutation import Mutation

class MyMutation(Mutation):
    def _do(self, problem, X, **kwargs):
        for i in range(X.shape[0]):
            X[i, :] = X[i, :]
            is_false = np.where(np.logical_not(X[i, :]))[0]
            is_true = np.where(X[i, :])[0]
            X[i, np.random.choice(is_false)] = True
            X[i, np.random.choice(is_true)] = False

        return X

In [755]:
class Problem(ElementwiseProblem):
    def __init__(self, data):
        super().__init__(n_var=len(data.columns.tolist()), n_obj=2)
        self.n_max = 2
        self.data = data

    def _evaluate(self, x, out, *args, **kwargs):
        clist = self.data.loc[:,x].columns.tolist()
        spread = cumret[clist[0]] - cumret[clist[1]]
        out["F"] = [ -spread.std(), cadf_pvalue(clist[0], clist[1], data)]

In [756]:
from pymoo.algorithms.moo.nsga2 import RankAndCrowdingSurvival
algorithm = NSGA2(
    pop_size=50,
    sampling=MySampling(),
    crossover=BinaryCrossover(),
    mutation=MyMutation(),
    survival=RankAndCrowdingSurvival(),
    eliminate_duplicates=True
)

In [757]:
problem = Problem(cumret)

In [758]:
res = minimize(problem,
               algorithm,
               ('n_gen', 80),
               seed=1,
               verbose=True)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |      5 |             - |             -
     2 |      100 |      5 |  0.2213833648 |         ideal
     3 |      150 |      8 |  0.0358153716 |         ideal
     4 |      200 |      5 |  0.0656081463 |             f
     5 |      250 |      6 |  0.1212718015 |         ideal
     6 |      300 |      6 |  0.0028507385 |         ideal
     7 |      350 |      6 |  0.2537884546 |         ideal
     8 |      400 |      6 |  0.0054771929 |             f
     9 |      450 |      6 |  0.000000E+00 |             f
    10 |      500 |      8 |  0.0505189964 |             f
    11 |      550 |      9 |  0.0132570095 |             f
    12 |      600 |     11 |  0.0415127204 |             f
    13 |      650 |     11 |  0.3003592083 |         ideal
    14 |      700 |     13 |  0.0251821630 |         nadir
    15 |      750 |     13 |  0.0019915351 |             f
    16 |      800 |     14 |  0.0048110500 |            

In [759]:
reslist = []
for item in res.X:
    reslist.append(data.loc[:,item].columns.tolist())
print("Pair                     Spread's SD                       CADF p-value")
for i in range(len(reslist)):
    print(f"{reslist[i]}            {-res.F[i][0]}              {res.F[i][1]}")

Pair                     Spread's SD                       CADF p-value
['BDX', 'CCL']            0.6829657059644704              0.00013689265895490827
['CCL', 'ENPH']            2.1014329939265015              0.2270307822871664
['AMD', 'BDX']            0.6850075449947801              0.0007076712005957271
['ES', 'VTRS']            0.5959639356931231              2.076898644176253e-07
['VTRS', 'WEC']            0.6142806969122736              3.6870499008243696e-07
['ENPH', 'NCLH']            1.9893898131798098              0.16513720907059304
['BDX', 'SEDG']            0.7717520985762042              0.001207559243966175
['BDX', 'TSLA']            1.1531817758323428              0.0012306319912815595
['ENPH', 'PNW']            1.5194433206225642              0.03768651522149427
['BDX', 'ENPH']            1.4816130767402031              0.001627687876033121
['AAL', 'ENPH']            1.9026583475705858              0.06199317264135702


# Testing For the Chosen Pairs

In [6]:
import extraction_process as ep
import kalman_spread_test as kst
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [7]:
# Get the original close data, cumulative return, and log return
data = ep.extract_and_process_data(0.2)
cumret = ep.metrics_process(data, "cumret")
logret = ep.metrics_process(data, 'logret')

# Get normalized logret
scaler = StandardScaler()
normal_logret = pd.DataFrame(scaler.fit_transform(logret), columns = data.columns, index = data.index).dropna(how='all')

Data Shape before cleaning = (1310, 503)
Data Shape after cleaning = (1310, 494)


In [8]:
perms = [['BDX', 'CCL'], ['CCL', 'ENPH'], ['AMD', 'BDX'], ['ES', 'VTRS'], ['VTRS', 'WEC'], ['ENPH', 'NCLH'], 
         ['BDX', 'SEDG'], ['BDX', 'TSLA'], ['ENPH', 'PNW'], ['BDX', 'ENPH'], ['AAL', 'ENPH'] ]

In [9]:
result = pd.DataFrame(columns = ['return_s1', 'return_s2', 'strategy_return'])
for perm in perms:
    print("For Pairs", perm)
    km_spread = kst.calculate_kalman_spread(cumret, perm[0], perm[1], '2022-01-01', '2022-12-31')
    return_s1, return_s2, enhanced_return = kst.backtest_pairs(data, km_spread, perm[0], perm[1], '2022-01-01', '2022-12-31')
    
    result.loc[f'{perm[0]}-{perm[1]} ', "return_s1"] = return_s1
    result.loc[f'{perm[0]}-{perm[1]} ', "return_s2"] = return_s2
    result.loc[f'{perm[0]}-{perm[1]} ', "strategy_return"] = enhanced_return
    print('\n')

For Pairs ['BDX', 'CCL']
Calculation for BDX - CCL Pairs
Buy-And-Hold Stratgies for BDX generates 4.36%
Buy-And-Hold Stratgies for CCL generates -62.35%
Pair Trading Stratgies for kalman spread generates -85.78%



For Pairs ['CCL', 'ENPH']
Calculation for CCL - ENPH Pairs
Buy-And-Hold Stratgies for CCL generates -62.35%
Buy-And-Hold Stratgies for ENPH generates 43.65%
Pair Trading Stratgies for kalman spread generates -70.53%



For Pairs ['AMD', 'BDX']
Calculation for AMD - BDX Pairs
Buy-And-Hold Stratgies for AMD generates -56.89%
Buy-And-Hold Stratgies for BDX generates 4.36%
Pair Trading Stratgies for kalman spread generates 87.00%



For Pairs ['ES', 'VTRS']
Calculation for ES - VTRS Pairs
Buy-And-Hold Stratgies for ES generates -3.09%
Buy-And-Hold Stratgies for VTRS generates -18.39%
Pair Trading Stratgies for kalman spread generates -55.64%



For Pairs ['VTRS', 'WEC']
Calculation for VTRS - WEC Pairs
Buy-And-Hold Stratgies for VTRS generates -18.39%
Buy-And-Hold Stratgies for 

In [10]:
result["evaluation"] = result["strategy_return"] - result["return_s1"] - result["return_s2"]
result

,return_s1,return_s2,strategy_return,evaluation
BDX-CCL,0.043643,-0.62354,-0.857805,-0.277908
CCL-ENPH,-0.62354,0.436487,-0.70531,-0.518256
AMD-BDX,-0.56889,0.043643,0.869977,1.395224
ES-VTRS,-0.03089,-0.183932,-0.556415,-0.341593
VTRS-WEC,-0.183932,0.00563,-0.594808,-0.416507
ENPH-NCLH,0.436487,-0.448152,-0.242505,-0.23084
BDX-SEDG,0.043643,0.002903,3.183759,3.137212
BDX-TSLA,0.043643,-0.691994,1.038275,1.686625
ENPH-PNW,0.436487,0.143005,1.443142,0.86365
BDX-ENPH,0.043643,0.436487,1.451357,0.971227
